<a href="https://colab.research.google.com/github/HLTT14/NLP-Assignments/blob/main/NLP_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gender Detection**

# Import Libraries

In [8]:
import os
import pandas as pd

# Import Data

In [15]:
data = []

def read_files(path, tag):
  os.chdir(path) 
  for file in os.listdir(): 
        with open(f"{path}/{file}", 'r') as f: 
            data.append([f.read(), tag])

read_files("/content/drive/MyDrive/Data sets/NLP Assignment1 Dataset/female", "female")
read_files("/content/drive/MyDrive/Data sets/NLP Assignment1 Dataset/male", "male")

df = pd.DataFrame(data, columns = ["Comment", "Gender"]) 
df

,Comment,Gender
0,من 4 روز میشه که از کیش اومدم وقتی وارد هتل شد...,female
1,ما هفته اول آبان ماه در آن هتل اقامت داشتیم و ...,female
2,ما پارسال این هتل بودیم\nاتاق تمیز و مرتب بود ...,female
3,باسلام من و مادرم از تاریخ 30تا2 درهتل انا بود...,female
4,من اول دیماه89 با دوتا از دوستام رفتم این هتل....,female
...,...,...
1995,من تا به حال دوبار به این هتل رفتم\nقیمت سوئیت...,male
1996,اقای ابی زاده دست مریضاد خسته نباشید ایول به ه...,male
1997,اگر به اسم 3 ستاره به این هتل برید کاملا در اش...,male
1998,درکل هتل خوبی بود البته هتل که نمبسه گفت بیشتر...,male
